## Workshop Week 6

## Logistic Regression
Breast Cancer data from [the UCI repository](http://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29) contains records corresponding to 
cases of observed tumors.   There are a number of observations for each and a categorisation in the `class` column: 2 for benign (good), 4 for malignant (bad).  Your task is to build a logistic regression model to classify these cases. 

The data is provided as a CSV file.  There are a small number of cases where no value is available, these are indicated in the data with `?`. I have used the `na_values` keyword for `read_csv` to have these interpreted as `NaN` (Not a Number).  Your first task is to decide what to do with these rows. You could just drop these rows or you could [impute them from the other data](http://scikit-learn.org/stable/modules/preprocessing.html#imputation-of-missing-values).

You then need to follow the procedure outlined in the lecture for generating a train/test set, building and evaluating a model. Your goal is to build the best model possible over this data.   Your first step should be to build a logistic regression model using all of the features that are available.
  

In [55]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import r2_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.feature_selection import RFE

In [42]:
bcancer = pd.read_csv("data/breast-cancer-wisconsin.csv", na_values="?")
bcancer.head()

,sample_code_number,clump_thickness,uniformity_cell_size,uniformity_cell_shape,marginal_adhesion,single_epithelial_cell_size,bare_nuclei,bland_chromatin,normal_nucleoli,mitoses,class
0,1000025,5,1,1,1,2,1.0,3,1,1,2
1,1002945,5,4,4,5,7,10.0,3,2,1,2
2,1015425,3,1,1,1,2,2.0,3,1,1,2
3,1016277,6,8,8,1,3,4.0,3,7,1,2
4,1017023,4,1,1,3,2,1.0,3,1,1,2


In [43]:
# Examine the data: check number of rows and number of columns
bcancer.shape

(699, 11)

In [44]:
# Look at the statistical summary of the dataframe
bcancer.describe()

,sample_code_number,clump_thickness,uniformity_cell_size,uniformity_cell_shape,marginal_adhesion,single_epithelial_cell_size,bare_nuclei,bland_chromatin,normal_nucleoli,mitoses,class
count,6.990000e+02,699.000000,699.000000,699.000000,699.000000,699.000000,683.000000,699.000000,699.000000,699.000000,699.000000
mean,1.071704e+06,4.417740,3.134478,3.207439,2.806867,3.216023,3.544656,3.437768,2.866953,1.589413,2.689557
std,6.170957e+05,2.815741,3.051459,2.971913,2.855379,2.214300,3.643857,2.438364,3.053634,1.715078,0.951273
min,6.163400e+04,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000
25%,8.706885e+05,2.000000,1.000000,1.000000,1.000000,2.000000,1.000000,2.000000,1.000000,1.000000,2.000000
50%,1.171710e+06,4.000000,1.000000,1.000000,1.000000,2.000000,1.000000,3.000000,1.000000,1.000000,2.000000
75%,1.238298e+06,6.000000,5.000000,5.000000,4.000000,4.000000,6.000000,5.000000,4.000000,1.000000,4.000000
max,1.345435e+07,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,4.000000


In [45]:
# Check how many classes we do have from the "class" column
bcancer["class"].nunique()

2

In [46]:
# Check number of samples for each class and comment whether dataset is balanced?
class_counts = bcancer['class'].value_counts()
print("Class counts:")
print(class_counts)

class_proportions = class_counts / class_counts.sum()
print("\nClass proportions:")
print(class_proportions)

Class counts:
2    458
4    241
Name: class, dtype: int64

Class proportions:
2    0.655222
4    0.344778
Name: class, dtype: float64


Since the difference in proportions between Class 2 and Class 4 is substantial, with Class 2 having a significantly larger proportion, the dataset is considered imbalanced. It is a 65% to 35%.

In [47]:
bcancer.dropna(inplace=True)

In [48]:
# Split your data into training(80%) and testing data (20%) and use random_state=142
# Convert class labels to binary (0: benign, 1: malignant)
bcancer["class"] = bcancer["class"].map({2: 0, 4: 1})

# Split the data into features (X) and labels (y)
X = bcancer.drop(["sample_code_number", "class"], axis=1)
y = bcancer["class"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=142)


In [49]:
# Build your Logistic Regression model
# Create and train the logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [53]:
# Do predictions on test set
y_pred = model.predict(X_test)

### Evaluation

To evaluate a classification model we want to look at how many cases were correctly classified and how many
were in error.  In this case we have two outcomes - benign and malignant.   SKlearn has some useful tools, the 
[accuracy_score]() function gives a score from 0-1 for the proportion correct.  The 
[confusion_matrix](http://scikit-learn.org/stable/modules/model_evaluation.html#confusion-matrix) function 
shows how many were classified correctly and what errors were made.  Use these to summarise the performance of 
your model (these functions have already been imported above).

In [57]:
# Evaluate the performance of your trained model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Confusion Matrix:\n", conf_matrix)

Accuracy: 0.9635036496350365
Confusion Matrix:
 [[83  2]
 [ 3 49]]


## Accuracy
The accuracy of your model is approximately 96.35%, which means that the model correctly predicted the class of 96.35% of the samples in the test set. While accuracy is a common metric, it might not always give the full picture, especially if the classes are imbalanced.

## Confusion Matrix
The confusion matrix provides detailed information about the predictions made by your model. It's a 2x2 matrix where rows represent the actual classes, and columns represent the predicted classes.

- The top-left value (83) represents the number of true negatives (TN). These are cases where the model correctly predicted a benign class (0) when the true class was also benign.
- The top-right value (2) represents the number of false positives (FP). These are cases where the model incorrectly predicted a malignant class (1) when the true class was benign (0).
- The bottom-left value (3) represents the number of false negatives (FN). These are cases where the model incorrectly predicted a benign class (0) when the true class was malignant (1).
- The bottom-right value (49) represents the number of true positives (TP). These are cases where the model correctly predicted a malignant class (1) when the true class was also malignant.

**This is the checkpoint mark for this week's workshop. You need to report `Accuracy Score` on test set and also show `confusion matrix`. You also need to provide analysis based on the results you got.**

### Feature Selection

Since you have many features available, one part of building the best model will be to select which features to use as input to the classifier. Your initial model used all of the features but it is possible that a better model can 
be built by leaving some of them out.   Test this by building a few models with subsets of the features - how do your models perform? 

This process can be automated.  The [sklearn RFE function](http://scikit-learn.org/stable/modules/feature_selection.html#recursive-feature-elimination) implements __Recursive Feature Estimation__ which removes 
features one by one, evaluating the model each time and selecting the best model for a target number of features.  Use RFE to select features for a model with 3, 4 and 5 features - can you build a model that is as good or better than your initial model?

In [58]:
for num_features in [3, 4, 5]:
    rfe_model = LogisticRegression()
    rfe_selector = RFE(estimator=rfe_model, n_features_to_select=num_features)
    rfe_selector = rfe_selector.fit(X_train, y_train)
    
    rfe_X_train = rfe_selector.transform(X_train)
    rfe_X_test = rfe_selector.transform(X_test)
    
    rfe_model.fit(rfe_X_train, y_train)
    rfe_y_pred = rfe_model.predict(rfe_X_test)
    rfe_accuracy = accuracy_score(y_test, rfe_y_pred)
    print(f"Model with {num_features} features Accuracy:", rfe_accuracy)

Model with 3 features Accuracy: 0.9708029197080292
Model with 4 features Accuracy: 0.9562043795620438
Model with 5 features Accuracy: 0.9635036496350365


## Conclusion

Write a brief conclusion to your experiment.  You might comment on the proportion of __false positive__ and __false negative__ classifications your model makes.  How useful would this model be in a clinical diagnostic setting? 

In a clinical diagnostic setting, the trade-off between false positives and false negatives is crucial. The consequences of false negatives can be severe, as delayed treatment can impact patient outcomes. Therefore, the model's performance on false negatives is of particular concern. The model's recall for the malignant class should be high to minimize false negatives.

Looking at the results, the model with 3 selected features achieved the highest accuracy and could potentially perform well in a clinical diagnostic setting. However, considering the higher recall achieved by the model with 4 features, it might be a better choice due to its ability to minimize false negatives.